# Initial Data Exploraation

In [1]:
import pandas as pd
import os
from pathlib import Path

# Set pandas display options for better output
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 20)

print("Libraries imported successfully")

Libraries imported successfully


In [2]:
# Load all CSV files
data_dir = Path('./data')
dataframes = {}

print("Loading CSV files:")
for csv_file in data_dir.glob('*.csv'):
    # Clean table name (remove olist_ and _dataset)
    table_name = csv_file.stem.replace('olist_', '').replace('_dataset', '')
    
    # Load CSV
    df = pd.read_csv(csv_file)
    dataframes[table_name] = df
    
    print(f"✓ {table_name}: {len(df):,} rows, {len(df.columns)} columns")

print(f"\nLoaded {len(dataframes)} tables successfully!")

Loading CSV files:
✓ sellers: 3,095 rows, 4 columns
✓ product_category_name_translation: 71 rows, 2 columns
✓ orders: 99,441 rows, 8 columns
✓ order_items: 112,650 rows, 7 columns
✓ customers: 99,441 rows, 5 columns
✓ order_items: 112,650 rows, 7 columns
✓ customers: 99,441 rows, 5 columns
✓ geolocation: 1,000,163 rows, 5 columns
✓ order_payments: 103,886 rows, 5 columns
✓ geolocation: 1,000,163 rows, 5 columns
✓ order_payments: 103,886 rows, 5 columns
✓ order_reviews: 99,224 rows, 7 columns
✓ products: 32,951 rows, 9 columns

Loaded 9 tables successfully!
✓ order_reviews: 99,224 rows, 7 columns
✓ products: 32,951 rows, 9 columns

Loaded 9 tables successfully!


In [3]:
# Examine each table structure
for table_name, df in dataframes.items():
    print(f"\n🔍 TABLE: {table_name.upper()}")
    print("="*50)
    print(f"Shape: {df.shape[0]:,} rows × {df.shape[1]} columns")
    print(f"Columns: {list(df.columns)}")
    print(f"Data types: {dict(df.dtypes)}")
    
    # Check for null values
    null_counts = df.isnull().sum()
    if null_counts.sum() > 0:
        print(f"Null values: {dict(null_counts[null_counts > 0])}")
    
    # Show sample data
    print("Sample data:")
    print(df.head(2))


🔍 TABLE: SELLERS
Shape: 3,095 rows × 4 columns
Columns: ['seller_id', 'seller_zip_code_prefix', 'seller_city', 'seller_state']
Data types: {'seller_id': dtype('O'), 'seller_zip_code_prefix': dtype('int64'), 'seller_city': dtype('O'), 'seller_state': dtype('O')}
Sample data:
                          seller_id  seller_zip_code_prefix seller_city  \
0  3442f8959a84dea7ee197c632cb2df15                   13023    campinas   
1  d1b65fc7debc3361ea86b5f14c68d2e2                   13844  mogi guacu   

  seller_state  
0           SP  
1           SP  

🔍 TABLE: PRODUCT_CATEGORY_NAME_TRANSLATION
Shape: 71 rows × 2 columns
Columns: ['product_category_name', 'product_category_name_english']
Data types: {'product_category_name': dtype('O'), 'product_category_name_english': dtype('O')}
Sample data:
    product_category_name product_category_name_english
0            beleza_saude                 health_beauty
1  informatica_acessorios         computers_accessories

🔍 TABLE: ORDERS
Shape: 99,441 r

In [4]:
# Data Analysis Summary for dbt Staging Strategy
print("DATA ANALYSIS SUMMARY FOR DBT STAGING MODELS")
print("-" * 60)

# Table sizes ranked by row count
print("\nTable Sizes (prioritized by row count):")
table_stats = [(name, len(df), len(df.columns)) 
               for name, df in dataframes.items()]
table_stats.sort(key=lambda x: x[1], reverse=True)

for name, rows, cols in table_stats:
    print(f"{name:<20} {rows:>8,} rows  {cols:>2} columns")

# Identify timestamp columns requiring type casting
print("\nTimestamp columns requiring ::timestamp casting:")
timestamp_cols = []
for table_name, df in dataframes.items():
    for col in df.columns:
        if any(keyword in col.lower() for keyword in ['date', 'timestamp', 'time']):
            timestamp_cols.append((table_name, col))

for table, col in timestamp_cols:
    print(f"{table}.{col}")

# Map ID columns for relationship analysis
print("\nID columns (for table relationships):")
id_mapping = {}
for table_name, df in dataframes.items():
    ids = [col for col in df.columns if col.endswith('_id')]
    if ids:
        id_mapping[table_name] = ids

for table, ids in id_mapping.items():
    print(f"{table:<20} {', '.join(ids)}")

# Identify data quality issues
print("\nColumns with significant null values (>10%):")
quality_issues = []
for table_name, df in dataframes.items():
    null_percentages = (df.isnull().sum() / len(df) * 100)
    high_nulls = null_percentages[null_percentages > 10]
    
    for col, pct in high_nulls.items():
        quality_issues.append(f"{table_name}.{col}: {pct:.1f}% null")

if quality_issues:
    for issue in quality_issues:
        print(issue)
else:
    print("No major data quality issues detected")

print("\n" + "-" * 60)
print("Staging Model Implementation Priority:")
print("1. Core entities: orders, products, customers")
print("2. Transaction details: order_items, order_reviews")  
print("3. Supporting data: sellers, order_payments")
print("4. Reference data: geolocation, product_category_name_translation")

DATA ANALYSIS SUMMARY FOR DBT STAGING MODELS
------------------------------------------------------------

Table Sizes (prioritized by row count):
geolocation          1,000,163 rows   5 columns
order_items           112,650 rows   7 columns
order_payments        103,886 rows   5 columns
orders                 99,441 rows   8 columns
customers              99,441 rows   5 columns
order_reviews          99,224 rows   7 columns
products               32,951 rows   9 columns
sellers                 3,095 rows   4 columns
product_category_name_translation       71 rows   2 columns

Timestamp columns requiring ::timestamp casting:
orders.order_purchase_timestamp
orders.order_delivered_carrier_date
orders.order_delivered_customer_date
orders.order_estimated_delivery_date
order_items.shipping_limit_date
order_reviews.review_creation_date
order_reviews.review_answer_timestamp

ID columns (for table relationships):
sellers              seller_id
orders               order_id, customer_id
order_